In [210]:
import numpy as np
import random
import math
import copy
import warnings
import time

# Create a calss of node
class Node:
    def __init__(self, parent, board, marker):
        self.parent = parent
        self.board = board
        self.children = []
        self.marker = marker
        self.n = 0
        self.q = 0
        self.ucb = 10000

def monte_carlo_tree_search(root,t):
    inter = 0
    while inter<=t:
        leaf = traverse2(root)
        simulation_result = rollout(leaf)
        #print(leaf.board)
        backpropagation(leaf,terminal(simulation_result))
        inter+=1  
    return best_child(root)

# Create an empty board given a size
def create_board(n):
    board = [' ']*n**2
    return board

# Find the index of the empty sapce on the board
def node_select(node):
    node_index = []
    for i in range(len(node.board)):
        if node.board[i] == ' ':
            node_index.append(i)
    return node_index

def play(node):
    choice = node_select(node)
    if choice != []:
        choice = random.choice(choice)
        if node.board.count('X') == node.board.count('O'):
            node.board[choice] = 'O'
        else:
            node.board[choice] = 'X'
    return node

# judge whether the state is terminal or not
def terminal(node):
    n = int(math.sqrt(len(node.board)))
    
    for i in range(n):
        if node.board[n*i:n*i+n] == ['O']*n:
            if node.marker == 'O':
                return 1
            else:
                return -1
        elif node.board[n*i:n*i+n] == ['X']*n:
            if node.marker == 'O':
                return -1
            else:
                return 0
        elif node.board[i:(n-1)*n+i+1:n] == ['O']*n:
            if node.marker == 'O':
                return 1
            else:
                return -1
        elif node.board[i:(n-1)*n+i+1:n] == ['X']*n:
            if node.marker == 'O':
                return -1
            else:
                return 1
    if node.board[0::n+1] == ['O']*n:
        if node.marker == 'O':
            return 1
        else:
            return -1
    elif node.board[0::n+1] == ['X']*n:
        if node.marker == 'O':
            return -1
        else:
            return 1
    elif node.board[n-1:-1:n-1] == ['O']*n:
        if node.marker == 'O':
            return 1
        else:
            return -1
    elif node.board[n-1:-1:n-1] == ['X']*n:
        if node.marker == 'O':
            return -1
        else:
            return 1 
    elif ' ' in node.board:
        return 10
    else:
        return 0 

# expand the selected node
def node_expand(node):
    index = node_select(node)
    if index != [] and terminal(node) == 10:
        for i in index:
            board = copy.deepcopy(node.board)
            node1 = Node(parent = node,board = board,marker = node.marker)
            if node1.board.count('X') == node1.board.count('O'):
                node1.board[i] = 'O'
            else:
                node1.board[i] = 'X'
            node.children.append(node1)
    return node

# find the node with best uct
def best_ucb(node): # the chosen node must has children, children != []
    list_ucb = []
    list_child = node.children
    for node_child in list_child:
        list_ucb.append(node_child.ucb)
    max_index = list_ucb.index(max(list_ucb))
    selected_node = list_child[max_index]   
    return selected_node

# judge whether a node is fully expanded
def fully_expanded(node):
    list_n = []
    for node_child in node.children:
        list_n.append(node_child.n)
    #print(np.prod(list_n))
    if np.prod(list_n) == 0 or list_n == []:
        return False
    else:
        return True

# select unvisited child, if the node is fully expanded, select child with highest uct as the new selected node    
def traverse2(node):
    if node.parent == None and node.n == 0:
        node = node_expand(node)
    while fully_expanded(node):
        node = best_ucb(node)
        if node.children == []:
            node = node_expand(node)
    if terminal(node) == 10:
        return best_ucb(node)
    else:
        return node

# rollout the node
def rollout(node):
    while terminal(node) == 10:
        node = rollout_policy(node)
    return node

# rollout policy --> random sampling
def rollout_policy(node):
    node1 = copy.deepcopy(node)
    node1 = play(node1)
    return node1

# backpropagate
def backpropagation(node, value):

    done = False

    #Update all parent nodes up to root node
    while done == False:
        node.n += 1
        node.q += value
        

        if node.parent == None:
            node.ucb = (node.q/node.n)
            done = True

        else:
            #if node.parent.n == 0: 
            node.parent.n += 1
            node.ucb = (node.q/node.n) + 2 * np.sqrt(np.log(node.parent.n)/node.n)
            node = node.parent

# find the best child which is most visited
def best_child(node):
    list_n = []
    list_child = node.children
    for node_child in list_child:
        list_n.append(node_child.n)
    index = list_n.index(max(list_n))
    return list_child[index]

In [211]:
warnings.filterwarnings("ignore")

def iteration_time(n):
    x = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    y = [500, 300, 200, 150, 100, 50, 40, 30, 10]
    z1 = np.polyfit(x, y, 10) 
    p1 = np.poly1d(z1) 
    return int(p1(n))
    
def terminal_judge(board):
    n = int(math.sqrt(len(board)))
    
    for i in range(n):
        if board[n*i:n*i+n] == ['O']*n:
            return 'Player O wins'
        elif board[n*i:n*i+n] == ['X']*n:
            return 'Player X wins'
        elif board[i:(n-1)*n+i+1:n] == ['O']*n:
            return 'Player O wins'
        elif board[i:(n-1)*n+i+1:n] == ['X']*n:
            return 'Player X wins'
    if board[0::n+1] == ['O']*n:
        return 'Player O wins'
    elif board[0::n+1] == ['X']*n:
        return 'Player X wins'
    elif board[n-1:-1:n-1] == ['O']*n:
        return 'Player O wins'
    elif board[n-1:-1:n-1] == ['X']*n:
        return 'Player X wins'
    elif ' ' in board:
        return 10
    else:
        return 'Tie'

def step_select(board):
    node_index = []
    for i in range(len(board)):
        if board[i] == ' ':
            node_index.append(i)
    return node_index
    
def opponent_play_X(board):
    choice = step_select(board)
    if choice != []:
        choice = random.choice(choice)
    board[choice] = 'X'
    return board

def opponent_play_O(board):
    choice = step_select(board)
    if choice != []:
        choice = random.choice(choice)
    board[choice] = 'O'
    return board

def draw_board(board):
    board = np.array(board).reshape(int(math.sqrt(len(board))),int(math.sqrt(len(board))))
    return board
    
def play_AIvsRandom_X(size):
    time_start = time.time()
    outcome = 10
    board = create_board(size)
    while outcome == 10:
        node = Node(None,board,'O')
        board = monte_carlo_tree_search(node,100).board
        board_state = draw_board(board)
        print('turn: O')
        print(board_state)
        if terminal_judge(board) != 10:
            outcome = terminal_judge(board)
            print(outcome)
            time_end = time.time()
            time_sum = time_end-time_start
            print(time_sum)
            return
        board = opponent_play_X(board)
        board_state = draw_board(board)
        print('turn: X')
        print(board_state)
        if terminal_judge(board) != 10:
            outcome = terminal_judge(board)
            print(outcome)
            time_end = time.time()
            time_sum = time_end-time_start
            print(time_sum)
            return

def play_AIvsRandom_O(size):
    time_start = time.time()
    outcome = 10
    board = create_board(size)
    n = 1
    while outcome == 10:
        board = opponent_play_O(board)
        board_state = draw_board(board)
        n += 1
        print('turn: O')
        print(board_state)
        if terminal_judge(board) != 10:
            outcome = terminal_judge(board)
            print(outcome)
            time_end = time.time()
            time_sum = time_end-time_start
            print(time_sum)
            return
        inter = iteration_time(n)
        node = Node(None,board,'X')
        board = monte_carlo_tree_search(node,100).board
        board_state = draw_board(board)
        n += 1
        print('turn: X')
        print(board_state)
        if terminal_judge(board) != 10:
            outcome = terminal_judge(board)
            print(outcome)
            time_end = time.time()
            time_sum = time_end-time_start
            print(time_sum)
            return        

def play_AIvsAI(size):
    time_start = time.time()
    outcome = 10
    board = create_board(size)
    n=1
    while outcome == 10:
        inter = iteration_time(n)
        node = Node(None,board,'O')
        board = monte_carlo_tree_search(node,100).board
        board_state = draw_board(board)
        n += 1
        print('turn: O')
        print(board_state)
        if terminal_judge(board) != 10:
            outcome = terminal_judge(board)
            print(outcome)
            time_end = time.time()
            time_sum = time_end-time_start
            print(time_sum)
            return
        inter = iteration_time(n)
        node = Node(None,board,'X')
        board = monte_carlo_tree_search(node,100).board
        board_state = draw_board(board)
        n += 1
        print('turn: X')
        print(board_state)
        if terminal_judge(board) != 10:
            outcome = terminal_judge(board)
            print(outcome)
            time_end = time.time()
            time_sum = time_end-time_start
            print(time_sum)
            return
        
play_AIvsAI(3)   

turn: O
[[' ' ' ' ' ']
 [' ' 'O' ' ']
 [' ' ' ' ' ']]
turn: X
[['X' ' ' ' ']
 [' ' 'O' ' ']
 [' ' ' ' ' ']]
turn: O
[['X' ' ' ' ']
 ['O' 'O' ' ']
 [' ' ' ' ' ']]
turn: X
[['X' ' ' ' ']
 ['O' 'O' 'X']
 [' ' ' ' ' ']]
turn: O
[['X' ' ' 'O']
 ['O' 'O' 'X']
 [' ' ' ' ' ']]
turn: X
[['X' ' ' 'O']
 ['O' 'O' 'X']
 ['X' ' ' ' ']]
turn: O
[['X' 'O' 'O']
 ['O' 'O' 'X']
 ['X' ' ' ' ']]
turn: X
[['X' 'O' 'O']
 ['O' 'O' 'X']
 ['X' 'X' ' ']]
turn: O
[['X' 'O' 'O']
 ['O' 'O' 'X']
 ['X' 'X' 'O']]
Tie
2.2285594940185547


In [236]:
def play_AIvsRandom_O_1(size):
    time_start = time.time()
    outcome = 10
    board = create_board(size)
    n = 1
    while outcome == 10:
        board = opponent_play_O(board)
        board_state = draw_board(board)
        n += 1
        if terminal_judge(board) != 10:
            outcome = terminal_judge(board)
            time_end = time.time()
            time_sum = time_end-time_start
            return outcome
        inter = iteration_time(n)
        node = Node(None,board,'X')
        board = monte_carlo_tree_search(node,100).board
        board_state = draw_board(board)
        n += 1
        if terminal_judge(board) != 10:
            outcome = terminal_judge(board)
            time_end = time.time()
            time_sum = time_end-time_start
            return outcome      

play_outcome = ['Player O wins','Player X wins','Tie']
play_cnt = [0, 0, 0]
outcome_dic = dict(zip(play_outcome,play_cnt))
for i in range(100):
    outcome = play_AIvsRandom_O_1(4)
    outcome_dic[outcome] +=1

In [235]:
 outcome_dic

{'Player O wins': 5, 'Player X wins': 58, 'Tie': 37}

In [230]:
def play_AIvsRandom_X_1(size):
    outcome = 10
    board = create_board(size)
    while outcome == 10:
        node = Node(None,board,'O')
        board = monte_carlo_tree_search(node,100).board
        board_state = draw_board(board)
        if terminal_judge(board) != 10:
            outcome = terminal_judge(board)
            return outcome
        board = opponent_play_X(board)
        board_state = draw_board(board)
        if terminal_judge(board) != 10:
            outcome = terminal_judge(board)
            return outcome
        
play_outcome1 = ['Player O wins','Player X wins','Tie']
play_cnt1 = [0, 0, 0]
outcome_dic1 = dict(zip(play_outcome1,play_cnt1))
for i in range(100):
    outcome1 = play_AIvsRandom_X_1(3)
    outcome_dic1[outcome1] +=1

In [231]:
outcome_dic1

{'Player O wins': 97, 'Player X wins': 2, 'Tie': 1}

In [239]:
def play_AIvsAI_1(size):
    outcome = 10
    board = create_board(size)
    n=1
    while outcome == 10:
        inter = iteration_time(n)
        node = Node(None,board,'O')
        board = monte_carlo_tree_search(node,100).board
        board_state = draw_board(board)
        n += 1
        if terminal_judge(board) != 10:
            outcome = terminal_judge(board)
            return outcome
        inter = iteration_time(n)
        node = Node(None,board,'X')
        board = monte_carlo_tree_search(node,100).board
        board_state = draw_board(board)
        n += 1
        if terminal_judge(board) != 10:
            outcome = terminal_judge(board)
            return outcome
        
play_outcome2 = ['Player O wins','Player X wins','Tie']
play_cnt2 = [0, 0, 0]
outcome_dic2 = dict(zip(play_outcome2,play_cnt2))
for i in range(100):
    outcome2 = play_AIvsAI_1(3)
    outcome_dic2[outcome2] +=1

In [240]:
outcome_dic2

{'Player O wins': 78, 'Player X wins': 12, 'Tie': 10}